In [104]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import json

import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection, AutoModelForCausalLM


In [105]:
#model used for captioning image input query
input_processor = AutoProcessor.from_pretrained("microsoft/git-base-coco")
input_model = AutoModelForCausalLM.from_pretrained("microsoft/git-base-coco").to("cuda")
#model used for object detection
dino_processor = AutoProcessor.from_pretrained("IDEA-Research/grounding-dino-tiny")
dino_model = AutoModelForZeroShotObjectDetection.from_pretrained("IDEA-Research/grounding-dino-tiny").to("cuda")

Use below for regular use

In [82]:
input_url = "input-data/traffic light.jpg"
image_url = "http://images.cocodataset.org/val2017/000000228942.jpg"

In [83]:
#opens the image input, and captions it
image = Image.open(input_url)

pixel_values = input_processor(images=image, return_tensors="pt").pixel_values.to("cuda")

generated_ids = input_model.generate(pixel_values=pixel_values, max_new_tokens=2)
generated_caption = input_processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(generated_caption)

image = Image.open(requests.get(image_url, stream=True).raw)
# Uses generated caption to detect
text_labels = [[generated_caption, "null"]]

inputs = dino_processor(images=image, text=text_labels, return_tensors="pt").to("cuda")
with torch.no_grad():
    outputs = dino_model(**inputs)

results = dino_processor.post_process_grounded_object_detection(
    outputs,
    inputs.input_ids,
    box_threshold=0.4,
    text_threshold=0.3,
    target_sizes=[image.size[::-1]]
)

full_results = []

# Retrieve the first image result
result = results[0]
for box, score, labels in zip(result["boxes"], result["scores"], result["labels"]):
    box = [round(x, 2) for x in box.tolist()]
    print(f"Detected {labels} with confidence {round(score.item(), 3)} at location {box}")
    full_results.append(box)

traffic light
Detected traffic light with confidence 0.455 at location [2.93, 18.95, 13.89, 37.75]
Detected null with confidence 0.436 at location [0.72, 131.54, 382.22, 299.81]


c:\ProgramData\miniconda3\Lib\site-packages\transformers\models\grounding_dino\processing_grounding_dino.py:95: FutureWarning: The key `labels` is will return integer ids in `GroundingDinoProcessor.post_process_grounded_object_detection` output since v4.51.0. Use `text_labels` instead to retrieve string object names.
  warnings.warn(self.message, FutureWarning)


Use below for COCO evaluation.

In [106]:
def requests_retry_session(
    retries=100,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

In [135]:
coco_dataset = open("data/instances_val2017.json")
coco_json = json.load(coco_dataset)
coco_images = coco_json["images"]
coco_annotations = coco_json["annotations"]
coco_categories = coco_json["categories"]
inputs_url = []
image_url = []
for images in coco_images:
    image_inputs = []
    for anno in coco_annotations:
        if anno["image_id"] == images["id"]:
            for categories in coco_categories:
                if anno["category_id"] == categories["id"]:
                    image_input = {
                        "input": "input-data/" + categories["name"] + ".jpg",
                        "category_id": categories["id"]
                    }
                    if image_input not in image_inputs:
                        image_inputs.append(image_input)
                    break
    inputs_url.append(image_inputs)
    image_url.append(images)

In [136]:
print(len(inputs_url))
print(len(image_url))

5000
5000


In [144]:
full_results = []
for x in range(len(inputs_url)):
    #opens the image input, and captions it
    image_descriptions = []
    for input_url in inputs_url[x]:
        image = Image.open(input_url["input"])

        pixel_values = input_processor(images=image, return_tensors="pt").pixel_values.to("cuda")

        generated_ids = input_model.generate(pixel_values=pixel_values, max_length=50)
        generated_caption = input_processor.batch_decode(generated_ids, skip_special_tokens=True)[0]


        image = Image.open(requests_retry_session().get(image_url[x]["coco_url"], stream=True).raw)
        image = image.convert('RGB')
        # Uses generated caption to detect
        text_labels = [generated_caption]

        inputs = dino_processor(images=image, text=text_labels, return_tensors="pt").to("cuda")
        with torch.no_grad():
            outputs = dino_model(**inputs)

        results = dino_processor.post_process_grounded_object_detection(
            outputs,
            inputs.input_ids,
            box_threshold=0.4,
            text_threshold=0.3,
            target_sizes=[image.size[::-1]]
        )


        # Retrieve the first image result
        for result in results:
            for box, score, labels in zip(result["boxes"], result["scores"], result["labels"]):
                box = [round(x, 2) for x in box.tolist()]
                formatted_results = {
                    "image_id": image_url[x]["id"],
                    "category_id": input_url["category_id"],
                    "bbox": box,
                    "score": round(score.item(), 3)
                }
                full_results.append(formatted_results)
    print(str(x + 1) + " out of " + str(len(image_url)))
full_results = json.dumps(full_results)

1 out of 5000


KeyboardInterrupt: 

In [ ]:
with open("results.json", "w") as f:
  f.write(full_results)